# UKWA Stats Module

Copyright 2023 Michael George (AKA Logiqx).

This file is part of [wsw-results](https://github.com/Logiqx/wsw-results) and is distributed under the terms of the GNU General Public License.

wsw-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

wsw-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

### Summary

Adhoc script to determine the correctness of UKWA flags in the current year

## Initialisation

Basic approach to determine the project directory

In [36]:
import os
import sys
import glob

import csv
import math

import numpy as np

from datetime import datetime

In [37]:
PROJDIR = os.path.realpath(os.path.join(sys.path[0], '..'))

In [38]:
RUN_2_KTS = 'Run 2 (kts)'
AVG_KTS = 'Avg (kts)'
STATUS = 'Status'

## Process Entrants

Process all names

In [39]:
def load_courses():
    '''Load courses'''
       
    csvFile = os.path.join(PROJDIR, 'resources', 'ukwa', 'analysis', 'courses.csv')
    
    courses = {}
    with open(csvFile, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for row in csvReader:
            courses[row['session']] = row['course']

    return courses

In [41]:
def isValidOld(results):
    '''Evaluate results using old rules'''
    
    allRuns = []
    for result in results:
        try:
            allRuns.append(float(result[RUN_2_KTS]))
        except ValueError:
            allRuns.append(0)
            
    allRuns = sorted(allRuns, reverse=True)
            
    percent25 = math.ceil(len(allRuns) / 4)
    percent50 = math.ceil(len(allRuns) / 2)
    
    valid = False

    if allRuns[percent25 - 1] >= 28:
        valid = True
        
    return valid
    

def isValidNew(results):
    '''Evaluate results using new rules'''
    
    allRuns = []
    for result in results:
        try:
            allRuns.append(float(result[AVG_KTS]))
        except ValueError:
            allRuns.append(0)
            
    allRuns = sorted(allRuns, reverse=True)
            
    percent25 = len(allRuns) // 4
    percent50 = len(allRuns) // 2
    
    valid = False

    if allRuns[percent25 - 1] >= 28:
        valid = True
    elif allRuns[percent25 - 1] >= 27.5 and allRuns[percent50 - 1] > 26:
        valid = True

    return allRuns[percent25 - 1], allRuns[percent50 - 1], valid
    

def isValidComplex(results, course, harbourSpeed=27, shoreSpeed=27, proGold=True):
    '''Evaluate results using new rules - complex'''

    silverRuns = []
    for result in results:
        if proGold or 'gold' in result[STATUS].lower() or 'pro' in result[STATUS].lower():
            try:
                silverRuns.append(float(result[RUN_2_KTS]))
            except ValueError:
                silverRuns.append(0)

    valid = False

    if len(silverRuns) > 0:
        silverRuns = sorted(silverRuns, reverse=True)

        percent25 = math.ceil(len(silverRuns) / 4)
        percent50 = math.ceil(len(silverRuns) / 2)

        if course == 'H':
            threshold = harbourSpeed
        else:
            threshold = shoreSpeed

        if silverRuns[percent25 - 1] >= threshold or silverRuns[percent50 - 1] >= threshold - 2:
            valid = True
        
    return valid

    
def processSession(year, csvFile, courses):
    '''processSession processes an individual session for an event'''

    results = []
    with open(csvFile, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for result in csvReader:
            results.append(result)

    session = os.path.basename(os.path.dirname(csvFile))
    course = courses[session]
    
    validOld = isValidOld(results)
    percent25, percent50, validNew = isValidNew(results)

    #valid27 = isValidComplex(results, course)
    #valid26 = isValidComplex(results, course, harbourSpeed=26, shoreSpeed=26)

    baseUrl = 'https://results.weymouthspeedweek.com/events'
    courseDesc = 'Harbour' if course == 'H' else 'Shore'
    link = '[{} / {} / {}]({}/{}/{}/ukwa.html)'.format(session, os.path.splitext(os.path.basename(csvFile))[0], courseDesc,
                                                     baseUrl, session[:4], session)

    if validNew is True and len(results) < 12:
        print('{} {},{},{}'.format(link, percent25, percent50, len(results)))

In [42]:
def processEvent(year, courses):
    '''processYear processes all of the sessions for an event'''

    csvFiles = sorted(glob.glob(os.path.join(PROJDIR, 'events', str(year), 'sailwave', '*', 'UKWA*Men*.csv')))
    
    for csvFile in csvFiles:
        if "Wing" not in csvFile:
            processSession(year, csvFile, courses)

In [43]:
def main():
    courses = load_courses()
    
    for year in range(2010, 2030):
        processEvent(year, courses)

    print('\nAll done')

if __name__ == '__main__':
    main()

[20131018 / UKWA - Men / Harbour](https://results.weymouthspeedweek.com/events/2013/20131018/ukwa.html) 31.59,28.28,10
[20151007 / UKWA - Leg 2 - Men / Shore](https://results.weymouthspeedweek.com/events/2015/20151007/ukwa.html) 30.12,26.7,10
[20191007 / UKWA - Leg 1 - Men / Harbour](https://results.weymouthspeedweek.com/events/2019/20191007/ukwa.html) 34.0,33.45,9
[20191007 / UKWA - Leg 2 - Men / Harbour](https://results.weymouthspeedweek.com/events/2019/20191007/ukwa.html) 34.36,33.69,9
[20191009 / UKWA - Leg 1 - Men / Shore](https://results.weymouthspeedweek.com/events/2019/20191009/ukwa.html) 32.89,32.24,11
[20221016 / UKWA - Leg 1 - Sailboards (Men) / Harbour](https://results.weymouthspeedweek.com/events/2022/20221016/ukwa.html) 29.19,25.67,11
[20221019 / UKWA - Leg 1 - Sailboards (Men) / Harbour](https://results.weymouthspeedweek.com/events/2022/20221019/ukwa.html) 31.35,26.04,11
[20221021 / UKWA - Leg 1 - Sailboards (Men) / Shore](https://results.weymouthspeedweek.com/events/202

## All Done!